# CS270 Final Project

#### David Gaddy, Samee Ibraheem, and Daniel Filan

## Overview

For our project, we implemented a dual decomposition algorithm for machine translation
alignments. The alignment process of machine translation tries to find words in one language
that correspond to words in another. Typical alignment models are asymmetric, so aligning
language 1 to language 2 is not the same as aligning language 2 to language 2, however we
would prefer if our alignments agreed in both directions. The paper *Model-Based Aligner
Combination Using Dual Decomposition* by John DeNero and Klaus Macherey uses a dual
decomposition algorithm to encourage agreement between the two directions. This algorithm
allows us to do effecient approximate inference while encouraging agreement by using dual
variables from a Lagragian relaxation.

## Implementation
The code for our implementation is the in the file `our_align_dual.py`.  In this jupyter notebook, we will demonstrate some results that we got with our implementation.

First, let us load a model we trained on an English-French parallel corpus of 10,000 sentences.

In [1]:
from our_align_dual import *

t_ef, q_ef = load_model('small_models_fr/ef_model.pkl.ibm1')
t_fe, q_fe = load_model('small_models_fr/fe_model.pkl.ibm1')

### Baseline

We have two models, one for each direction English->French and French->English.  Without dual decomposition, we can use each of these directions separately or take the intersection of the aligments suggested by each directional model.

In [2]:
dev_files = {'en':'fr_data/en-fr.en', 'f':'fr_data/en-fr.fr'}
gold_file = 'fr_data/en-fr.align'
reverse_output = False

en_sent = load_sentences(dev_files['en'])
f_sent = load_sentences(dev_files['f'])

dir1_alignments = []
dir2_alignments = []
intersection_alignments = []
union_alignments = []
num_intersection = 0
num_union = 0
for i in range(len(en_sent)):
    # aligns each foreign word to the english one
    align_ef = align_ibm(en_sent[i], f_sent[i], t_ef, q_ef)
    align_ef = filter_null_alignments([(ei,fi) for fi,ei in enumerate(align_ef)])
    # aligns each english word to a foreign one
    align_fe = align_ibm(f_sent[i], en_sent[i], t_fe, q_fe)
    align_fe = filter_null_alignments([(ei,fi) for ei,fi in enumerate(align_fe)])

    dir1_alignments.append(align_ef)
    dir2_alignments.append(align_fe)

    intersection = combine_intersect(align_ef, align_fe)
    union = combine_union(align_ef, align_fe)
    num_intersection += len(intersection)
    num_union += len(union)
    intersection_alignments.append(intersection)
    union_alignments.append(union)
print('intersection over union', num_intersection / num_union)

save_alignments(dir1_alignments, 'ibm-align-dir1.txt', reverse_output)
save_alignments(dir2_alignments, 'ibm-align-dir2.txt', reverse_output)
save_alignments(intersection_alignments, 'ibm-align-intersection.txt', reverse_output)
save_alignments(union_alignments, 'ibm-align-union.txt', reverse_output)
print('direction 1')
!perl measure-alignment-error.pl fr_data/en-fr.align ibm-align-dir1.txt
print('direction 2')
!perl measure-alignment-error.pl fr_data/en-fr.align ibm-align-dir2.txt
print('intersection')
!perl measure-alignment-error.pl fr_data/en-fr.align ibm-align-intersection.txt


intersection over union 0.4040338321405335
direction 1
Ref=1009	Test=1135	MatchRef=627	MatchTest=691
Prec=60.88	Rec=62.14	AER=38.50
direction 2
Ref=1009	Test=1023	MatchRef=612	MatchTest=674
Prec=65.88	Rec=60.65	AER=36.84
intersection
Ref=1009	Test=621	MatchRef=534	MatchTest=561
Prec=90.34	Rec=52.92	AER=33.25


The two numbers we will focus on are labeled *intersection over union* and *AER*.  Intersection over union is a metric for how much the alignments agree between the two directions.  Higher is better, and a value of 1 would indicate perfect agreement.  AER, or alignment error rate, is a measure of how much the final alignments agree with alignments annotated by people who know both languages.  Lower AER is better.  The intersected alignment baseline gets an AER of 33.25%.

### Dual Decomposition

Dual decomposition is a method of encouraging alignments from the two different model directions to agree.  It uses Lagrange multipliers to enforce the agreement.  These Lagrange multipliers end up being combined with the model in a way that allows us to run the same inference procedure we would typically use but with slightly modified model probabilities.

Let's look at how dual decomposition affects our results:

In [4]:
alignments = []
num_converged = 0
num_intersection = 0
num_union = 0
for i in range(len(en_sent)):
    intersection, union, converged = align_dual(en_sent[i], f_sent[i], t_ef, t_fe, q_ef, q_fe, 1, 250)
    intersection = filter_null_alignments(intersection)
    union = filter_null_alignments(union)

    num_intersection += len(intersection)
    num_union += len(union)
    alignments.append(intersection)
    if converged:
        num_converged += 1
print('intersection over union',num_intersection / num_union)
print('converged',float(num_converged) / len(en_sent))

save_alignments(alignments, 'dual-decomp-intersection.txt', reverse_output)
!perl measure-alignment-error.pl fr_data/en-fr.align dual-decomp-intersection.txt


intersection over union 0.5144927536231884
converged 0.02
Ref=1009	Test=781	MatchRef=574	MatchTest=623
Prec=79.77	Rec=56.89	AER=33.59


We can see that the intersection over union increased significantly, which means that we were successful in encouraging agreement between the two directions.  The AER is slightly worse, though, which means increasing agreement didn't cause the alignments to agree more with human annotations.

One thing to note is that the convergence rate, or percentage of time where the two directions completely agree after dual decomposition, is very low.  Dual decomposition is an iterative algorithm, and given infinite time, it should converge 100% of the time.  However, the convergence can be very slow, and we cut off the algorithm after 250 iterations.  Even without having converged, though, we can see that it has significantly increased agreement.